# Installing Necessary Libraries

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

# Loading and Preparing the Model and the Tokenizer

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 512
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.6: Fast Qwen2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Prompt Engineering Techniques

In [3]:
from transformers import TextStreamer

## Zero-Shot Prompting

In [4]:
prompt_zero_shot = "Generate a Python function to calculate the Fibonacci sequence up to n."

inputs = tokenizer(prompt_zero_shot, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)

Generate a Python function to calculate the Fibonacci sequence up to n. def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        fib = [0, 1]
        for i in range(2, n):
            fib.append(fib[i-1] + fib[i-2])
        return fib<|endoftext|>


## Few-Shot Prompting

In [5]:
prompt_few_shot = """
# Example 1:
# Input: Write a Python function to reverse a list
# Output:
def reverse_list(lst):
    return lst[::-1]

# Example 2:
# Input: Write a Python function to check if a number is prime
# Output:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Now, write a Python function to calculate the Fibonacci sequence up to n and generate only the code for this task without any more examples.
"""

inputs = tokenizer(prompt_few_shot, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


# Example 1:
# Input: Write a Python function to reverse a list
# Output:
def reverse_list(lst):
    return lst[::-1]

# Example 2:
# Input: Write a Python function to check if a number is prime
# Output:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Now, write a Python function to calculate the Fibonacci sequence up to n and generate only the code for this task without any more examples.
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]<|endoftext|>


## Chain-of-Thought Prompting

In [6]:
prompt_chain_of_thought = """
Write a Python function to calculate the factorial of a number. Here’s how we can break down the steps:
1. Define a function that takes an integer input.
2. Use a loop to multiply all integers from 1 up to that number.
3. Return the product.

Now, generate the Python code for this.
"""

inputs = tokenizer(prompt_chain_of_thought, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


Write a Python function to calculate the factorial of a number. Here’s how we can break down the steps:
1. Define a function that takes an integer input.
2. Use a loop to multiply all integers from 1 up to that number.
3. Return the product.

Now, generate the Python code for this.
```python
def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
```
This function calculates the factorial of a given number by iterating from 1 to n and multiplying each number to the result.<|endoftext|>


## Role-Playing Prompt

In [7]:
prompt_role_playing = """
You are a helpful Python Programming Assistant, skilled in writing clean and efficient code. Your task is to help a user by generating Python functions based on their requirements.

The user will ask you to write specific functions, and you will respond with only the Python code that solves the problem. Do not provide any explanations or extra text. Just the code. Here's the first task:

Task: Write a Python function to calculate the nth Fibonacci number using dynamic programming.

User: Please write a function to calculate the nth Fibonacci number using dynamic programming.
"""

inputs = tokenizer(prompt_role_playing, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id)


You are a helpful Python Programming Assistant, skilled in writing clean and efficient code. Your task is to help a user by generating Python functions based on their requirements.

The user will ask you to write specific functions, and you will respond with only the Python code that solves the problem. Do not provide any explanations or extra text. Just the code. Here's the first task:

Task: Write a Python function to calculate the nth Fibonacci number using dynamic programming.

User: Please write a function to calculate the nth Fibonacci number using dynamic programming.
```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        fib = [0, 1]
        for i in range(2, n + 1):
            fib.append(fib[i - 1] + fib[i - 2])
        return fib[n]
```<|endoftext|>


## Self-Consistency Prompting

In [8]:
prompt_self_consistency = """
Generate two different Python functions for finding the maximum number in a list:
1. Using a for loop.
2. Using the max() function.

Then, print the result of each to verify they produce the same output.
"""

inputs = tokenizer(prompt_self_consistency, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


Generate two different Python functions for finding the maximum number in a list:
1. Using a for loop.
2. Using the max() function.

Then, print the result of each to verify they produce the same output.
"""
# Function 1: Using a for loop
def find_max_for_loop(lst):
    max_num = lst[0]
    for num in lst:
        if num > max_num:
            max_num = num
    return max_num

# Function 2: Using the max() function
def find_max_max_function(lst):
    return max(lst)

# Test the functions
test_list = [10, 20, 30, 40, 50]
print("Maximum number using for loop:", find_max_for_loop(test_list))
print("Maximum number using max() function:", find_max_max_function(test_list))<|endoftext|>


## Prompt with Instructions and Constraints

In [9]:
prompt_with_constraints = """
Create a Python function to compute the factorial of a number with the following constraints:
1. **Function Name**: The function should be named `optimized_factorial`.
2. **Optimization Requirement**: Implement it recursively but avoid redundant calculations by using a cache (memoization).
3. **Error Handling**: The function should handle invalid inputs (e.g., negative numbers) by raising a `ValueError` with an informative message.
4. **Docstring**: Include a docstring that explains the function's purpose, parameters, return type, and error cases.
5. **Type Hinting**: Use Python type hints to specify that the function accepts an integer and returns an integer.

Generate the code based on these instructions.
"""

inputs = tokenizer(prompt_with_constraints, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


Create a Python function to compute the factorial of a number with the following constraints:
1. **Function Name**: The function should be named `optimized_factorial`.
2. **Optimization Requirement**: Implement it recursively but avoid redundant calculations by using a cache (memoization).
3. **Error Handling**: The function should handle invalid inputs (e.g., negative numbers) by raising a `ValueError` with an informative message.
4. **Docstring**: Include a docstring that explains the function's purpose, parameters, return type, and error cases.
5. **Type Hinting**: Use Python type hints to specify that the function accepts an integer and returns an integer.

Generate the code based on these instructions.
```python
def optimized_factorial(n: int) -> int:
    """
    Compute the factorial of a non-negative integer n using recursion with memoization.
    
    Parameters:
    n (int): A non-negative integer whose factorial is to be computed.
    
    Returns:
    int: The factorial of 

## Few-Shot with Step-by-Step Breakdown

In [10]:
prompt_few_shot_step_by_step = """
# Example 1:
# Task: Write a Python function to check if a string is a palindrome.
# Step-by-Step Breakdown:
# Step 1: Define a function named `is_palindrome` that accepts a string.
# Step 2: Convert the string to lowercase to ensure case insensitivity.
# Step 3: Compare the string with its reverse.
# Step 4: Return True if they match, otherwise return False.
def is_palindrome(s: str) -> bool:
    s = s.lower()
    return s == s[::-1]

# Example 2:
# Task: Write a Python function to find the GCD of two numbers using the Euclidean algorithm.
# Step-by-Step Breakdown:
# Step 1: Define a function named `gcd` that accepts two integers.
# Step 2: Use a loop to repeatedly divide and replace until the remainder is zero.
# Step 3: Return the result when the remainder reaches zero.
def gcd(a: int, b: int) -> int:
    while b:
        a, b = b, a % b
    return a

# Task: Now, write a Python function to calculate the nth Fibonacci number and generate only the code for this task without any more examples.
# Step-by-Step Breakdown:
# Step 1: Define a function named `fibonacci` that accepts an integer `n`.
# Step 2: Implement the function recursively.
# Step 3: Use memoization to store results for previously computed numbers to optimize.
# Step 4: Return the nth Fibonacci number.
"""

inputs = tokenizer(prompt_few_shot_step_by_step, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


# Example 1:
# Task: Write a Python function to check if a string is a palindrome.
# Step-by-Step Breakdown:
# Step 1: Define a function named `is_palindrome` that accepts a string.
# Step 2: Convert the string to lowercase to ensure case insensitivity.
# Step 3: Compare the string with its reverse.
# Step 4: Return True if they match, otherwise return False.
def is_palindrome(s: str) -> bool:
    s = s.lower()
    return s == s[::-1]

# Example 2:
# Task: Write a Python function to find the GCD of two numbers using the Euclidean algorithm.
# Step-by-Step Breakdown:
# Step 1: Define a function named `gcd` that accepts two integers.
# Step 2: Use a loop to repeatedly divide and replace until the remainder is zero.
# Step 3: Return the result when the remainder reaches zero.
def gcd(a: int, b: int) -> int:
    while b:
        a, b = b, a % b
    return a

# Task: Now, write a Python function to calculate the nth Fibonacci number and generate only the code for this task without any more

## Combined Prompt with Multiple Approaches

In [11]:
prompt_combined = """
You are a highly skilled Python Programming Assistant capable of generating efficient and well-structured Python code. The user will request a solution, and you are to follow these specific instructions:
1. **Zero-shot approach**: When the task is clear and no examples are given, generate the code based purely on your knowledge.
2. **Few-shot approach**: If the user provides some examples or context, use them to generate better solutions while adhering to the pattern established in those examples.
3. **Prompt with instructions and constraints**: Follow the instructions carefully, respecting the constraints of the problem.
4. **Role-playing**: Act as if you are a friendly assistant helping the user with their Python code, offering the best solutions based on the task at hand.

The task is as follows:
**Task**: Write a Python function that calculates the factorial of a number. The function must handle edge cases, like negative inputs, and should be optimized for performance using memoization.

**Instructions**:
- The function must be named `factorial`.
- If the input is a negative number, return `None` to indicate that the factorial cannot be computed.
- Use memoization to optimize the function for repeated calculations.
- Do not add any additional explanations or text. Only provide the Python code.

**Examples**:
Example 1: Calculate the factorial of a positive number:
factorial(5)
# Expected output: 120

Example 2: Handle negative numbers gracefully:
factorial(-1)
# Expected output: None

Your response should strictly follow the instructions provided and include only the Python code for the factorial function. Do not add any other comments or explanations.
"""

inputs = tokenizer(prompt_combined, return_tensors="pt")
text_streamer = TextStreamer(tokenizer)

FastLanguageModel.for_inference(model)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)


You are a highly skilled Python Programming Assistant capable of generating efficient and well-structured Python code. The user will request a solution, and you are to follow these specific instructions:
1. **Zero-shot approach**: When the task is clear and no examples are given, generate the code based purely on your knowledge.
2. **Few-shot approach**: If the user provides some examples or context, use them to generate better solutions while adhering to the pattern established in those examples.
3. **Prompt with instructions and constraints**: Follow the instructions carefully, respecting the constraints of the problem.
4. **Role-playing**: Act as if you are a friendly assistant helping the user with their Python code, offering the best solutions based on the task at hand.

The task is as follows:
**Task**: Write a Python function that calculates the factorial of a number. The function must handle edge cases, like negative inputs, and should be optimized for performance using memoiz